In [2]:
from tools import get_puzzle, show_problem_1, show_problem_2

TODAY = 20
puzzle = get_puzzle(TODAY)
show_problem_1(puzzle)

https://adventofcode.com/2023/day/20
## --- Day 20: Pulse Propagation ---


With your help, the Elves manage to find the right parts and fix all of the machines. Now, they just need to send the command to boot up the machines and get the sand flowing again.


The machines are far apart and wired together with long **cables** . The cables don't connect to the machines directly, but rather to communication **modules** attached to the machines that perform various initialization tasks and also act as communication relays.


Modules communicate using **pulses** . Each pulse is either a **high pulse** or a **low pulse** . When a module sends a pulse, it sends that type of pulse to each module in its list of **destination modules** .


There are several different types of modules:


 **Flip-flop** modules (prefix%``) are either **on** or **off** ; they are initially **off** . If a flip-flop module receives a high pulse, it is ignored and nothing happens. However, if a flip-flop module receives a low pulse, it **flips between on and off** . If it was off, it turns on and sends a high pulse. If it was on, it turns off and sends a low pulse.


 **Conjunction** modules (prefix&``) **remember** the type of the most recent pulse received from **each** of their connected input modules; they initially default to remembering a **low pulse** for each input. When a pulse is received, the conjunction module first updates its memory for that input. Then, if it remembers **high pulses** for all inputs, it sends a **low pulse** ; otherwise, it sends a **high pulse** .


There is a single **broadcast module** (namedbroadcaster``). When it receives a pulse, it sends the same pulse to all of its destination modules.


Here at Desert Machine Headquarters, there is a module with a single button on it called, aptly, the **button module** . When you push the button, a single **low pulse** is sent directly to thebroadcaster``module.


After pushing the button, you must wait until all pulses have been delivered and fully handled before pushing it again. Never push the button if modules are still processing pulses.


Pulses are always processed **in the order they are sent** . So, if a pulse is sent to modulesa``,b``, andc``, and then modulea``processes its pulse and sends more pulses, the pulses sent to modulesb``andc``would have to be handled first.


The module configuration (your puzzle input) lists each module. The name of the module is preceded by a symbol identifying its type, if any. The name is then followed by an arrow and a list of its destination modules. For example:


```
 broadcaster -> a, b, c
 %a -> b
 %b -> c
 %c -> inv
 &inv -> a

```


In this module configuration, the broadcaster has three destination modules nameda``,b``, andc``. Each of these modules is a flip-flop module (as indicated by the%``prefix).a``outputs tob``which outputs toc``which outputs to another module namedinv``.inv``is a conjunction module (as indicated by the&``prefix) which, because it has only one input, acts like aninverter(it sends the opposite of the pulse type it receives); it outputs toa``.


By pushing the button once, the following pulses are sent:


```
 button -low-> broadcaster
 broadcaster -low-> a
 broadcaster -low-> b
 broadcaster -low-> c
 a -high-> b
 b -high-> c
 c -high-> inv
 inv -low-> a
 a -low-> b
 b -low-> c
 c -low-> inv
 inv -high-> a

```


After this sequence, the flip-flop modules all end up **off** , so pushing the button again repeats the same sequence.


Here's a more interesting example:


```
 broadcaster -> a
 %a -> inv, con
 &inv -> b
 %b -> con
 &con -> output

```


This module configuration includes thebroadcaster``, two flip-flops (nameda``andb``), a single-input conjunction module (inv``), a multi-input conjunction module (con``), and an untyped module namedoutput``(for testing purposes). The multi-input conjunction modulecon``watches the two flip-flop modules and, if they're both on, sends a **low pulse** to theoutput``module.


Here's what happens if you push the button once:


```
 button -low-> broadcaster
 broadcaster -low-> a
 a -high-> inv
 a -high-> con
 inv -low-> b
 con -high-> output
 b -high-> con
 con -low-> output

```


Both flip-flops turn on and a low pulse is sent tooutput``! However, now that both flip-flops are on andcon``remembers a high pulse from each of its two inputs, pushing the button a second time does something different:


```
 button -low-> broadcaster
 broadcaster -low-> a
 a -low-> inv
 a -low-> con
 inv -high-> b
 con -high-> output

```


Flip-flopa``turns off! Now,con``remembers a low pulse from modulea``, and so it sends only a high pulse tooutput``.


Push the button a third time:


```
 button -low-> broadcaster
 broadcaster -low-> a
 a -high-> inv
 a -high-> con
 inv -low-> b
 con -low-> output
 b -low-> con
 con -high-> output

```


This time, flip-flopa``turns on, then flip-flopb``turns off. However, beforeb``can turn off, the pulse sent tocon``is handled first, so it **briefly remembers all high pulses** for its inputs and sends a low pulse tooutput``. After that, flip-flopb``turns off, which causescon``to update its state and send a high pulse tooutput``.


Finally, witha``on andb``off, push the button a fourth time:


```
 button -low-> broadcaster
 broadcaster -low-> a
 a -low-> inv
 a -low-> con
 inv -high-> b
 con -high-> output

```


This completes the cycle:a``turns off, causingcon``to remember only low pulses and restoring all modules to their original states.


To get the cables warmed up, the Elves have pushed the button1000``times. How many pulses got sent as a result (including the pulses sent by the button itself)?


In the first example, the same thing happens every time the button is pushed:8``low pulses and4``high pulses are sent. So, after pushing the button1000``times,8000``low pulses and4000``high pulses are sent. Multiplying these together gives32000000``.


In the second example, after pushing the button1000``times,4250``low pulses and2750``high pulses are sent. Multiplying these together gives11687500``.


Consult your module configuration; determine the number of low pulses and high pulses that would be sent after pushing the button1000``times, waiting for all pulses to be fully handled after each push of the button. **What do you get if you multiply the total number of low pulses sent by the total number of high pulses sent?** 




In [14]:
import queue

ON = HIGH = 1
OFF = LOW = 0
DEBUG = False

class Module(object):
    module_type = "[mod]"
    def __init__(self, id, actions) -> None:
        self.id = id
        self.actions = actions
        self.receivers = []
        
    
    def setReceivers(self,receivers):
        self.receivers.extend(receivers)
    
    def reviewReceivers(self):
        
        for r in self.receivers:
            if r.module_type == "[&]": # we need to initialize its memory
                r.memory[self.id] = LOW
    
    def _send(self, signal):
        signal_str = "low"
        if signal == HIGH:
            signal_str = "high"

        if DEBUG: print(f"\t\t {self.id} will now send {signal_str}-> {[x.id for x in self.receivers]}")
        for receiver in self.receivers:
            self.actions.put ( [self, signal, receiver] )    

    def __repr__(self):
        return self.id + f"({self.module_type})"
    
    def pulse (self, signal, sender):
        pass #print(f"{self.id} received pulse {signal} from {sender.id} ")


class Flip_flop (Module):
    module_type = "[%]"


    def __init__(self, id, actions) -> None:
        super().__init__(id, actions)
        self.state = OFF

    def pulse (self, signal, sender):
        assert signal in [ON,OFF]
        if signal == HIGH:
            pass
        else:
            if self.state == ON:
                self.state = OFF
                self._send(LOW) 
            else:
                self.state = ON
                self._send(HIGH)
        

class Conjunction (Module):
    module_type = "[&]"

    def setReceivers(self,receivers):
        self.memory = {}
        super().setReceivers(receivers)
        
        
    
    def pulse (self, signal, sender):
        """if it remembers high pulses for all inputs, 
        it sends a low pulse; otherwise, it sends a high pulse."""
        if DEBUG: print(f"\t{self.id} {self.module_type} received {signal} \t\t{self.memory}")
        self.memory[sender.id] = signal
        for value in self.memory.values():
            if value == LOW:
                if DEBUG: print(f"\tand sends HIGH")
                self._send(HIGH)
                return
        if DEBUG: print(f"\tand sends LOW")
        self._send(LOW)

class Broadcast (Module):
    module_type = "Broadcast"
    def pulse (self, signal, sender):
        self._send(signal)



class Button (Module):
    module_type = "Button"
    def pulse (self, signal, sender):
        self._send(signal)




def solution_1(data, times):
    actions = queue.Queue()
    objects = {}

    # We create the objects
    button = Button("button", actions)
    output = Module("output", actions)
    objects["output"] = output
    
    rx = Module("rx", actions)
    objects["rx"] = rx

    for line in data:
        obj, receivers = line.split(" -> ")
        if obj[0] == '%':
            objects[obj[1:]] = Flip_flop(obj[1:], actions)
        elif obj[0] == '&':
            objects[obj[1:]] = Conjunction(obj[1:], actions)
        elif obj == 'broadcaster':
            objects["broadcaster"] = Broadcast("broadcaster", actions)
        else:
            print (f"found {obj} !!!!!!!!!!!!!!")

    print(objects)
    # We assign the receivers

    for line in data:
        obj, receivers = line.split(" -> ")
        
        if obj[0] == '%' or obj[0] == '&':
            objects[obj[1:]].setReceivers([objects[recv.strip()] for recv in receivers.split(",")])
        
        elif obj == 'broadcaster':
            objects["broadcaster"] .setReceivers([objects[recv.strip()] for recv in receivers.split(",")])
        
        else:
            print (f"found {obj} !!!!!!!!!!!!!!")   

    for obj in objects.values():
        obj.reviewReceivers()

    highs = 0
    lows = 0
    for i in range(times):
        first_action = [button, LOW, objects["broadcaster"]]
        actions. put(first_action)

        while not actions.empty():

            sender, signal, receiver = actions.get()

            signal_str = "low"
            if signal == HIGH:
                highs +=1
                signal_str = "high"
            else:
                lows +=1
                
            if DEBUG: print(f"{sender.id} -{signal_str}-> {receiver.id}                qsize: {actions.qsize()}")
            receiver.pulse(signal, sender)
    
    return highs * lows

test2 = """broadcaster -> a
%a -> inv, con
&inv -> b
%b -> con
&con -> output""".splitlines()

assert solution_1 (puzzle.test, 1000) == 32000000
assert solution_1 (test2, 1000) == 11687500
print( f"Solution 1:  {solution_1 (puzzle.data,1000)} is the product of the number of low pulses times the number of high pulses sent ")





{'output': output([mod]), 'rx': rx([mod]), 'broadcaster': broadcaster(Broadcast), 'a': a([%]), 'b': b([%]), 'c': c([%]), 'inv': inv([&])}
{'output': output([mod]), 'rx': rx([mod]), 'broadcaster': broadcaster(Broadcast), 'a': a([%]), 'inv': inv([&]), 'b': b([%]), 'con': con([&])}
{'output': output([mod]), 'rx': rx([mod]), 'rp': rp([%]), 'kh': kh([&]), 'jz': jz([%]), 'dx': dx([%]), 'dh': dh([%]), 'zv': zv([&]), 'xb': xb([%]), 'hv': hv([%]), 'db': db([%]), 'sk': sk([&]), 'tc': tc([%]), 'dj': dj([%]), 'jk': jk([%]), 'fm': fm([%]), 'dp': dp([%]), 'vh': vh([%]), 'lz': lz([&]), 'kr': kr([%]), 'jb': jb([%]), 'kz': kz([%]), 'ts': ts([%]), 'gr': gr([%]), 'kc': kc([%]), 'jd': jd([%]), 'bs': bs([%]), 'zk': zk([%]), 'vf': vf([%]), 'mm': mm([%]), 'qc': qc([%]), 'fk': fk([%]), 'bm': bm([%]), 'ds': ds([%]), 'sn': sn([%]), 'zn': zn([%]), 'ct': ct([%]), 'np': np([%]), 'tg': tg([&]), 'tx': tx([%]), 'zl': zl([%]), 'zz': zz([%]), 'ms': ms([%]), 'ns': ns([%]), 'px': px([%]), 'broadcaster': broadcaster(Broad

In [4]:
show_problem_2(puzzle)

## --- Part Two ---


The final machine responsible for moving the sand down to Island Island has a module attached namedrx``. The machine turns on when a **single low pulse** is sent torx``.


Reset all modules to their default states. Waiting for all pulses to be fully handled after each button press, **what is the fewest number of button presses required to deliver a single low pulse to the module namedrx``?** 




In [13]:
# used Wade's spoiler:
"""
rx works like the output module in the second example 
where it only receives pulses.

Looking at the input there's only one module 
that sends pulses to rx and it's a conjunction that has 4 inputs.
So for it to send a low pulse all 4 of its inputs need 
to send high pulses in the same button press.

To find when those happen keep track of what button press 
each of them send high pulses. Make sure 
when you are counting button presses you start at 1 not 0! 
Once you know the cycle length for each input 
you can find the answer by multiplying the cycle lengths together. 
I did the lowest common multiple 
since I had the function for that from a previous day but 
it wasn't necessary since my answer was just the product of the lengths.
"""
# for the quick hack we need to modify Conjunction and stop execution
# when any of the 4 & modules send a low pulse
class Conjunction (Module):
    module_type = "[&]"

    def setReceivers(self,receivers):
        self.memory = {}
        super().setReceivers(receivers)
        
        
    
    def pulse (self, signal, sender):
        """ These are the obtained periods:
        tg = 3769
        kh = 3889
        lz = 3917
        hn = 4013

        Solution is = 4013 * 3917 * 3769 * 3889"""

        if (self.id == 'kh' or
            self.id == 'lz' or 
             self.id == 'tg' or
              self.id == 'hn') and signal == 0:
            
            raise Exception (" it was " +self.id)        
        """if it remembers high pulses for all inputs, 
        it sends a low pulse; otherwise, it sends a high pulse."""
        if DEBUG: print(f"\t{self.id} {self.module_type} received {signal} \t\t{self.memory}")
        self.memory[sender.id] = signal
        for value in self.memory.values():
            if value == LOW:
                if DEBUG: print(f"\tand sends HIGH")
                self._send(HIGH)
                return
        if DEBUG: print(f"\tand sends LOW")
        self._send(LOW)

def solution_2(data):
    actions = queue.Queue()
    objects = {}

    # We create the objects
    button = Button("button", actions)
    output = Module("output", actions)
    objects["output"] = output
    
    rx = Module("rx", actions)
    objects["rx"] = rx

    for line in data:
        obj, receivers = line.split(" -> ")
        if obj[0] == '%':
            objects[obj[1:]] = Flip_flop(obj[1:], actions)
        elif obj[0] == '&':
            objects[obj[1:]] = Conjunction(obj[1:], actions)
        elif obj == 'broadcaster':
            objects["broadcaster"] = Broadcast("broadcaster", actions)
        else:
            raise Exception ("something's wrong 1!")

    # We assign the receivers

    for line in data:
        obj, receivers = line.split(" -> ")
        
        if obj[0] == '%' or obj[0] == '&':
            objects[obj[1:]].setReceivers([objects[recv.strip()] for recv in receivers.split(",")])
        
        elif obj == 'broadcaster':
            objects["broadcaster"] .setReceivers([objects[recv.strip()] for recv in receivers.split(",")])
        
        else:
            raise Exception ("something's wrong 2!")

    for obj in objects.values():
        obj.reviewReceivers()

    counter = 1
    while True:
        if counter % 1000 == 0:
            print (counter)
        try:
            first_action = [button, LOW, objects["broadcaster"]]
            actions. put(first_action)

            while not actions.empty():

                sender, signal, receiver = actions.get()

                signal_str = "low"
                if signal == HIGH:
                    signal_str = "high"
                    
                if DEBUG: print(f"{sender.id} -{signal_str}-> {receiver.id}                qsize: {actions.qsize()}")
                receiver.pulse(signal, sender)
            counter += 1
        except Exception as e:
            print(counter )
            print(e.args)
            return 4013 * 3917 * 3769 * 3889 # all 4 periods


print( f"Solution 1:  {solution_2 (puzzle.data)} is the fewest number of button presses required to deliver a single low pulse to the module named rx ")


{'output': output([mod]), 'rx': rx([mod]), 'rp': rp([%]), 'kh': kh([&]), 'jz': jz([%]), 'dx': dx([%]), 'dh': dh([%]), 'zv': zv([&]), 'xb': xb([%]), 'hv': hv([%]), 'db': db([%]), 'sk': sk([&]), 'tc': tc([%]), 'dj': dj([%]), 'jk': jk([%]), 'fm': fm([%]), 'dp': dp([%]), 'vh': vh([%]), 'lz': lz([&]), 'kr': kr([%]), 'jb': jb([%]), 'kz': kz([%]), 'ts': ts([%]), 'gr': gr([%]), 'kc': kc([%]), 'jd': jd([%]), 'bs': bs([%]), 'zk': zk([%]), 'vf': vf([%]), 'mm': mm([%]), 'qc': qc([%]), 'fk': fk([%]), 'bm': bm([%]), 'ds': ds([%]), 'sn': sn([%]), 'zn': zn([%]), 'ct': ct([%]), 'np': np([%]), 'tg': tg([&]), 'tx': tx([%]), 'zl': zl([%]), 'zz': zz([%]), 'ms': ms([%]), 'ns': ns([%]), 'px': px([%]), 'broadcaster': broadcaster(Broadcast), 'hn': hn([&]), 'hh': hh([%]), 'kf': kf([%]), 'vg': vg([%]), 'bv': bv([%]), 'pl': pl([&]), 'cm': cm([%]), 'cc': cc([%]), 'bf': bf([%]), 'hl': hl([%]), 'cs': cs([&]), 'gq': gq([%]), 'rg': rg([%]), 'sd': sd([&])}
1000
2000
3000
3769
(' it was tg',)
Solution 1:  23040230092536